In [1]:
import urllib.request as req
import os.path
import json
import requests
import os
from datetime import datetime, timedelta
import time
import pytz
from dateutil import parser
import pandas as pd
API_ROOT = "https://api.synopticdata.com/v2/"
API_TOKEN = "622863d185934c71a58c12a6e679c23d"
api_arguments = {"token":API_TOKEN}
NWS_TOKEN = "WkXIizXZDpEHxorFvkahwHGgBoDfFxRR"
SMU_API_TOKEN = 'cff7722463364a7ab147d188e0945c25'
import pickle
scaler = pickle.load(open('scaler.sav' , 'rb'))
model = pickle.load(open('storm_clustering.sav', 'rb'))

In [2]:
t_df = pd.DataFrame()
i = -1
while True:
    
    now = time.strftime('%Y%m%d%H%M', time.localtime())
    url = 'https://api.synopticdata.com/v2/stations/precip?stid={}&recent={}&precip=1&units=precip|in&hfmetar=1&obtimezone=local&token={}'.format('khou',60,SMU_API_TOKEN)
    print(url)
    req = requests.get(url)
    res = json.loads(req.text)
    if res['SUMMARY']['RESPONSE_CODE'] == 2 : 
        print(res)
    if res['SUMMARY']['RESPONSE_CODE'] == 1 :
        print('True')
        index = res['STATION'][0]['OBSERVATIONS']['ob_end_time_1']
        print(index)
        dt = pd.DataFrame(res['STATION'][0]['OBSERVATIONS'] , index=[i])
        dt['query_time'] = now
        print(now)
        t_df = t_df.append(dt)
        t_df['cumsum'] = t_df['total_precip_value_1'].cumsum()
        t_df['max_int'] = t_df['cumsum'].diff().fillna(0)

        intensity = max(t_df['max_int'])
        print('intensity = ',intensity)
        total_depth = t_df.iloc[len(t_df)-1]['cumsum']
        print('total_depth', total_depth)
        #print(t_df)
        if total_depth > 0:
            X = scaler.transform([[intensity , total_depth]])
            y = model.predict(X)
            if y == 5:
                y = 2

            t_df = t_df.set_index(t_df['ob_end_time_1'].apply(lambda x: parser.parse(x.split('T')[0].replace('-' , '/') + ' '+x.split('T')[1].split('-')[0])))
            if (y == 2) & (t_df['total_precip_value_1'].rolling('3h').max()[len(t_df)-1]==0):
                i=-1
                t_df = pd.DataFrame()
            elif (y == 1) & (t_df['total_precip_value_1'].rolling('5h').max()[len(t_df)-1]==0):
                i = -1
                t_df = pd.DataFrame()
            elif t_df['total_precip_value_1'].rolling('8h').max()[len(t_df)-1]==0:
                i = -1
                t_df = pd.DataFrame()
            else:
                i += 1    
        else:
            y = 'Rainless'
            i=-1
            t_df = pd.DataFrame()
            print(y)
    time.sleep(15*60)
 

https://api.synopticdata.com/v2/stations/precip?stid=khou&recent=60&precip=1&units=precip|in&hfmetar=1&obtimezone=local&token=cff7722463364a7ab147d188e0945c25
True
2020-09-20T14:53:00-0500
202009201508
intensity =  0.0
total_depth 0.22


AttributeError: 'AgglomerativeClustering' object has no attribute 'predict'